<a href="https://colab.research.google.com/github/CarpeDiemer/data-science/blob/master/Trump_union_of_state_speech_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-04-14 08:56:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-04-14 08:56:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  50.5MB/s    in 14s     

2019-04-14 08:56:54 (59.2 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [25]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [15]:
from urllib import *
from bs4 import BeautifulSoup

def get_only_text(url):
    """ 
    return the title and the text of the article
    at the specified url
    """
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
  
    print ("=====================")
    print (text)
    print ("=====================")
 
    return soup.title.text, text    
 
     
url="https://www.whitehouse.gov/briefings-statements/remarks-president-trump-state-union-address-2/"
text = get_only_text(url)


				Remarks			 
Issued on:
February 6, 2019
 February 5, 2019
9:07 P.M. EST THE PRESIDENT:  Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States — (applause) — and my fellow Americans: We meet tonight at a moment of unlimited potential.  As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans. Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.  (Applause.) The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.  It’s the agenda of the American people. Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and 

In [22]:
sentences = []
for s in text:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]
sentences[30:40]

['An amazing quality of life for all of our citizens is within reach.',
 'We can make our communities safer, our families stronger, our culture richer, our faith deeper, and our middle class bigger and more prosperous than ever before.',
 '(Applause.)',
 'But we must reject the politics of revenge, resistance, and retribution, and embrace the boundless potential of cooperation, compromise, and the common good.',
 '(Applause.)',
 'Together, we can break decades of political stalemate.',
 'We can bridge old divisions, heal old wounds, build new coalitions, forge new solutions, and unlock the extraordinary promise of America’s future.',
 'The decision is ours to make.',
 'We must choose between greatness or gridlock, results or resistance, vision or vengeance, incredible progress or pointless destruction.',
 'Tonight, I ask you to choose greatness.']

In [0]:
word_embeddings = {}
f = open('glove.6B.100d.txt',encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:],dtype='float32')
  word_embeddings[word] = coefs
f.close()

In [0]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')

In [0]:
new_sens = []
for sen in clean_sentences:
  sentence = []
  for word in sen.split():
    if word not in stop_words:
      sentence.append(word)
  new_sens.append(sentence)
clean_sentences = new_sens

In [37]:
word_embeddings.get('reward',np.zeros((100,)))

array([ 0.70715  ,  0.10404  , -0.072744 ,  0.025057 ,  0.24732  ,
        0.12195  , -0.27894  , -0.077417 , -0.23198  ,  0.3638   ,
        0.18672  ,  0.56909  , -0.31635  , -0.38125  , -0.18907  ,
        0.17741  ,  0.36154  , -0.53358  , -0.42781  ,  1.2119   ,
       -0.16373  , -0.52632  , -0.8      , -0.33105  ,  0.12048  ,
        0.48224  , -0.0086637,  0.061027 ,  0.057805 , -0.92399  ,
        0.36383  ,  0.32277  , -0.35765  , -0.0079183,  0.52382  ,
       -0.26712  , -0.37957  ,  0.45036  , -0.74361  , -0.27602  ,
        0.27345  , -0.031007 ,  0.32246  , -0.60293  ,  0.28225  ,
       -0.22959  ,  0.38223  , -0.32357  , -0.37521  , -0.68256  ,
        0.17264  , -0.5802   ,  0.61082  ,  1.0044   , -0.007259 ,
       -0.63536  ,  0.5433   , -0.9777   ,  1.1399   , -0.58679  ,
        0.12483  ,  0.058851 , -0.62514  ,  0.058948 ,  0.20573  ,
       -0.29362  , -0.47402  ,  0.55835  ,  0.67563  , -0.37842  ,
        0.1973   , -0.39399  , -0.43955  , -0.42417  ,  0.1357

In [0]:
sentence_vectors = []
for sen in clean_sentences:
  if len(sen) != 0:
    v = sum([word_embeddings.get(w,np.zeros((100,))) for w in sen]) / (len(sen))
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

sim_mat = np.zeros([len(sentences),len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)

In [43]:
ranked_sentences

[(0.0025909183336828927,
  'After 24 months of rapid progress, our economy is the envy of the world, our military is the most powerful on Earth, by far, and America — (applause) — America is again winning each and every day.'),
 (0.0025656162679008024,
  'Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.'),
 (0.002539124951222221,
  'I hope you can pass the USMCA into law so that we can bring back our manufacturing jobs in even greater numbers, expand American agriculture, protect intellectual property, and ensure that more cars are proudly stamped with our four beautiful words: “Made in the USA.”\xa0 (Applause.)'),
 (0.0025263216571922427,
  'When Grace completed treatment last fall, her doctors and nurses cheered — they loved her; they still love her — with tears in their eyes as she hung up a poster that read: “Last day of chemo.”\xa0 (Applause.)'),
 (0.002524252244494597,
  'We do no

In [45]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.summarization import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

2019-04-14 10:06:02,083 : WARNING : paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
2019-04-14 10:06:02,242 : INFO : 'pattern' package not found; tag filters are not available for English


In [48]:
!pip install sumy

    100% |████████████████████████████████| 81kB 5.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built breadability


In [0]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [50]:
LANGUAGE = "english"
SENTENCES_COUNT = 10
url="https://www.whitehouse.gov/briefings-statements/remarks-president-trump-state-union-address-2/"
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
print ("--LsaSummarizer--")    
summarizer = LsaSummarizer()
summarizer = LsaSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LsaSummarizer--
In June, we mark 75 years since the start of what General Dwight D. Eisenhower called the “Great Crusade” — the Allied liberation of Europe in World War II.
Now is the time for Congress to show the world that America is committed to ending illegal immigration and putting the ruthless coyotes, cartels, drug dealers, and human traffickers out of business.
Meanwhile, working-class Americans are left to pay the price for mass illegal migration: reduced jobs, lower wages, overburdened schools, hospitals that are so crowded you can’t get in, increased crime, and a depleted social safety net.
Tens of thousands of innocent Americans are killed by lethal drugs that cross our border and flood into our cities, including meth, heroin, cocaine, and fentanyl.
In the last two years, our brave ICE officers made 266,000 arrests of criminal aliens, including those charged or convicted of nearly 100,000 assaults, 30,000 sex crimes, and 4,000 killings or murders.
I hope you can pass the 

In [51]:
print ("--LuhnSummarizer--")     
summarizer = LuhnSummarizer() 
summarizer = LuhnSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this")
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LuhnSummarizer--
Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first.
Human traffickers and sex traffickers take advantage of the wide-open areas between our ports of entry to smuggle thousands of young girls and women into the United States and to sell them into prostitution and modern-day slavery.
Elvin says that, “If I can make sure these young girls get their justice, I’ve [really] done my job.” Thanks to his work, and that of his incredible colleagues, more than 300 women and girls have been rescued from the horror of this terrible situation, and more than 1,500 sadistic traffickers have been put behind bars.
The border city of El Paso, Texas used to have extre

In [52]:
print ("--LexRankSummarizer--")   
summarizer = LexRankSummarizer()
summarizer = LexRankSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this")
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LexRankSummarizer--
Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first.
Tonight, I am asking you to defend our very dangerous southern border out of love and devotion to our fellow citizens and to our country.
And thank you for being here.
When Elvin — (applause) — thank you.
To build on — (applause) — thank you.
I have great respect for President Xi, and we are now working on a new trade deal with China.
But we must do more.
As we make progress in these negotiations, we will be able to reduce our troop’s presence and focus on counterterrorism.
It’s time.
It’s the Americans!” (Applause.)


In [53]:
print ("--EdmundsonSummarizer--")     
summarizer = EdmundsonSummarizer() 
words1 = ("economy", "fight", "trade", "china")
summarizer.bonus_words = words1
     
words2 = ("another", "and", "some", "next")
summarizer.stigma_words = words2
    
words3 = ("another", "and", "some", "next")
summarizer.null_words = words3
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--EdmundsonSummarizer--
February 5, 2019
9:07 P.M. EST
We meet tonight at a moment of unlimited potential.
As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans.
Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.
(Applause.)
The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.
It’s the agenda of the American people.
There is a new opportunity in American politics, if only we have the courage, together, to seize it.
Tonight, I am also asking you to join me in another fight that all Americans can get behind: the fight against childhood cancer.
END 10:29 P.M. EST
